In [3]:
hd_dir = "/home/hpc/cudahub/tracer/hybriddomain/"
modelFileName = 'problems/logistic_delays'
conn_name = "dglconnection"
device_conf_name = "dgllogisticdevices"

In [4]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
if hd_dir not in sys.path:
    sys.path.insert(0, hd_dir)
env_dir = os.path.join(hd_dir, 'spaces', 'math_space', "common")
if env_dir not in sys.path:
    sys.path.insert(0, env_dir)
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from envs.hs.model.model_main import ModelNet as Model
from settings.settings_main import Settings

In [ ]:
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name, pf_prefix=currentdir)

In [ ]:
#block0 = model.blocks[0]
#equation = model.equations[0]
#model.paramValues

In [ ]:
from solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
progress = ProgressNotebook(100, prefix='solving')
from solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))
def run(event):
    remoteProjectRun(settings, 1, progress)
bTest.on_click(run)
display(progress.progress)
display(bTest)

### Results:

In [5]:
import numpy as np
import re
import matplotlib.pyplot as plt
from ipywidgets import interactive

out_dir = os.path.join(currentdir,  modelFileName, "out")
listdir = os.listdir(out_dir)

print(listdir)

outs = [os.path.join(out_dir, file_name)
        for file_name in listdir
        if '.out' in file_name]
print(outs)
with open(outs[0]) as f:
    data = f.read()

['.ipynb_checkpoints', 'logistic_delays-res0-U.out', 'logistic_delays-plot0.mp4']
['/home/hpc/cudahub/dgl/problems/logistic_delays/out/logistic_delays-res0-U.out']


In [6]:
lines = data.split('\n')
result = {}

for line in lines:
    if ':' in line:
        key, val = line.split(':')
        result[key]=val[1:]
    else:
        val = line
        result[key] += val

In [7]:
def gen(reuslt):
    for key in reuslt:
        step_0 = re.subn(r'\.\s+',".0,",result[key])[0]
        step_1 = re.subn(r'\s+',",",step_0)[0]
        step_2 = step_1.replace('.]','.0]')
        step_3 = step_2.replace('[,','[')

        yield((key, eval(step_3)))

In [12]:

result_t = dict([(float(key), val) for key, val in gen(result)])
time = [key for key in result_t]
result_x = np.array([result_t[key] for key in result_t]).T

##### U(x, t-fixed):

In [9]:
%matplotlib inline
def f(t):
    plt.plot(result_t[time[t]])
w = interactive(f,t=(0,100,1))
display(w)

interactive(children=(IntSlider(value=50, description='t'), Output()), _dom_classes=('widget-interact',))

##### U(x-fixed, t):

In [10]:
%matplotlib inline
def f(x):
    plt.plot(time, result_x[x])
w = interactive(f,x=(0,360,1))
display(w)

interactive(children=(IntSlider(value=180, description='x', max=360), Output()), _dom_classes=('widget-interac…